In [1]:
import gzip
import json
import pandas as pd
import os
import requests

tweets = []
with open("./filtered") as f:
    for line in f.readlines():
        tweets.append(json.loads(line))
        
dataset = pd.DataFrame(tweets)
print len(dataset)

# Create additional columns
dataset['screen_name'] = dataset.user.map(lambda x: x['screen_name'].lower())
dataset['name'] = dataset.user.map(lambda x: x['name'].lower())
dataset['statuses_count'] = dataset.user.map(lambda x: x['statuses_count'])
dataset['followers_count'] = dataset.user.map(lambda x: x['followers_count'])
dataset['text_lower'] = dataset.text.apply(lambda x: x.lower())

418


In [7]:
# guess country from coodrinates
def get_reverse_geocoding(coordinates):
    response = requests.get("http://api.geonames.org/countryCode?lng="
                           + str(coordinates[0]) + "&lat=" + str(coordinates[1]) + "&username=a01200356")
    text = response.text
    if len(text) > 4:
        return None
    return text[:2]

tweets_with_coordinates = dataset.dropna(subset=['coordinates'])
tweets_with_coordinates['country'] = tweets_with_coordinates.coordinates.map(lambda x: get_reverse_geocoding(x['coordinates']))
tweets_with_coordinates = tweets_with_coordinates.dropna(subset=['country'])
# print tweets_with_coordinates['country']

# get country from place
tweets_with_place = dataset.dropna(subset=['place'])
tweets_with_place['country'] = tweets_with_place.place.map(lambda x: x['country_code'])
# print tweets_with_place['country']

# get country from user location
tweets_with_location = dataset
tweets_with_location['country'] = tweets_with_location.user.map(lambda x: x['location'])
# print tweets_with_place['country']

# merge resulting datasets
tweets_with_country = pd.concat([tweets_with_coordinates, tweets_with_place], join='outer')
tweets_with_country = pd.concat([tweets_with_coordinates, tweets_with_location], join='outer')
tweets_with_country = tweets_with_country.dropna(subset=['country'])
# print tweets_with_country[['screen_name' ,'country']]

# count
country_counts = tweets_with_country[['country', 'text_lower']].groupby('country').agg('count').rename(columns={'text_lower': 'count'}).sort('count', ascending=False)

pd.set_option('display.max_rows', 1000)
print country_counts

                                                count
country                                              
worldwide                                          24
San Mateo, CA                                      20
Chicago, Global Headquarters                       18
Worldwide                                          12
Vancouver, British Columbia                         6
Cincinnati, Ohio (HQ)                               6
Atlanta, GA                                         5
World Wide                                          4
New York, NY                                        4
Plano, TX                                           4
United Kingdom                                      4
India                                               3
Canada                                              3
New York                                            3
Star,AB                                             3
Schenectady, N.Y.                                   3
Global                      

/home/marc/Programmation/ge-olympics/env/lib/python2.7/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/marc/Programmation/ge-olympics/env/lib/python2.7/site-packages/ipykernel/__main__.py:32: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
